### 1. 데이터 작업하기

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [3]:
# 공개 데이터셋에서 학습 데이터를 내려받기
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받기
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor(),
)

In [4]:
print(training_data, '\n')
print(test_data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor() 

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [5]:
batch_size = 64

# 데이터로더 생성
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
	print("Shape of X [N, C, H, W]: ", X.shape)
	print("Shape of y: ", y.shape, y.dtype)
	print("Example of y: ", y)
	break  # 하나의 데이터만 추출

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64
Example of y:  tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1,
        2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5])


### 2. 모델 만들기
- \_\_init\_\_ 함수 : 신경망 layer 정의
- forward 함수 : 신경망에 데이터를 어떻게 전달할지 지정

In [13]:
# 학습에 사용할 CPU나 GPU 장치 얻기
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# 모델 정의
class NeuralNetwork(nn.Module):
	def __init__(self):
		super(NeuralNetwork, self).__init__()  # super()로 기반 클래스의 __init__ 메서드 호출
		self.flatten = nn.Flatten()
		self.linear_relu_stack = nn.Sequential(
			nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
		)

	def forward(self, x):
		x = self.flatten(x)
		logits = self.linear_relu_stack(x)
		return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 3. 모델 매개변수 최적화하기

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [15]:
def train(dataloader, model, loss_fn, optimizer):
	size = len(dataloader.dataset)  		     # 60,000
	for batch, (X, y) in enumerate(dataloader):  # range(938)(=math.ceil(600000/64))
		X, y = X.to(device), y.to(device)

		# 예측 오류 계산
		pred = model(X)
		loss = loss_fn(pred, y)

		# 역전파
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		if batch % 100 == 0:
			loss, current = loss.item(), batch * len(X)  # len(X) : 64
			print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [54]:
def test(dataloader, model, loss_fn):
	size = len(dataloader.dataset)   # 10,000
	num_batches = len(dataloader)    # 157(=math.ceil(10000/64))
	test_loss, correct = 0, 0
	with torch.no_grad():
		for X, y in dataloader:      # range(157)
			X, y = X.to(device), y.to(device)
			pred = model(X)
			test_loss += loss_fn(pred, y).item()
			correct += (pred.argmax(1) == y).type(torch.float).sum().item()  # batch 내에 있는 모든 샘플들의 정답 일치 여부 확인      
	test_loss /= num_batches
	correct /= size
	print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n")

#### ★ test 코드 부연 설명

In [16]:
# device = torch.device("cuda")

def test(dataloader, model, loss_fn):
    number_of_samples = len(dataloader.dataset)
    number_of_batches = len(dataloader)
    
    list_of_average_batch_loss = []
    number_of_correct_predictions = 0
    
    with torch.no_grad():
        for batch_input, batch_label in dataloader:
            batch_input, batch_label = batch_input.to(device), batch_label.to(device)
            batch_prediction = model(batch_input)
            
            #★ Batch 안에 있는 각 sample data들의 loss를 구하고, 그 평균값을 average_loss_of_current_batch에 저장
            average_loss_of_current_batch = loss_fn(batch_prediction, batch_label).item()
            
            #★ 이번 batch에 있는 데이터들에 대해서 평균을 한 loss들을 list에 저장
            list_of_average_batch_loss.append(average_loss_of_current_batch)
            
            is_correct_prediction = (batch_prediction.argmax(dim=1)==batch_label)
            number_of_correct_predictions += is_correct_prediction.sum().item()
            
    #★ 평균 batch loss들을 모아 놓은 list에 대해 다시 평균하면, 전체 데이터들을 한 번에 평균 낸 것과 동일 = test_loss
    average_loss_of_all_data = sum(list_of_average_batch_loss) / number_of_batches
    ratio_of_correct_predictions = number_of_correct_predictions / number_of_samples
            

In [17]:
epochs= 5
for t in range(epochs):
	print(f"Epoch {t+1}\n------------------------------")
	train(train_dataloader, model, loss_fn, optimizer)
	test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------------
loss: 2.309907 [    0/60000]
loss: 2.297539 [ 6400/60000]
loss: 2.279222 [12800/60000]
loss: 2.271118 [19200/60000]
loss: 2.251609 [25600/60000]
loss: 2.236799 [32000/60000]
loss: 2.241691 [38400/60000]
loss: 2.211391 [44800/60000]
loss: 2.204928 [51200/60000]
loss: 2.175111 [57600/60000]
Done!


### 4. 모델 저장하기

In [56]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### 5. 모델 불러오기

In [57]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [98]:
classes = [
	"T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
	pred = model(x)  
	predicted, actual = classes[pred[0].argmax(0)], classes[y]  
	print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
